<a href="https://www.kaggle.com/code/mikedelong/reddit-comments-need-word-clouds?scriptVersionId=152166251" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/public-opinion-on-climate-change-updated-daily/reddit_opinion_climate_change.csv',
                index_col=['comment_id'], parse_dates=['created_time', 'post_created_time']).drop(columns=['downs', 'post_total_awards_received'])
df.head()

,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,user_is_verified,user_account_created_time,...,user_awarder_karma,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_created_time
comment_id,,,,,,,,,,,,,,,,,,,,,
kai3359,1,"I’m a climate scientist—not a physicist, geolo...",climatechange,2023-11-23 23:41:17,181i90d,torrentialwx,0,1,True,2020-08-22 04:34:29,...,240.0,17.0,19340.0,19993.0,66,https://www.reuters.com/business/environment/e...,Maybe a reason why people ignore climate models,0.89,66,2023-11-22 20:02:59
kai2yma,1,Guys like this are clearly not paying enough t...,EverythingScience,2023-11-23 23:40:16,181wg0q,workerbotsuperhero,0,1,True,2013-11-24 23:17:48,...,0.0,393152.0,182078.0,579736.0,1022,NaN,Centibillionaire Jeff Bezos' $500 million Koru...,0.97,1022,2023-11-23 08:16:12
kai2ybu,1,Humans are going to end ourselves. Extinction ...,climate,2023-11-23 23:40:12,18279vq,Big-Summer-,0,1,True,2023-08-21 17:04:57,...,0.0,81.0,8049.0,8140.0,213,NaN,New report says oil and gas sector must slash ...,0.98,213,2023-11-23 18:11:16
kai2lti,1,"BTW, US Total Debt now stands at an astonishin...",climateskeptics,2023-11-23 23:37:19,182da47,Left_Insanity,0,1,True,2022-03-06 00:23:51,...,0.0,21715.0,14123.0,36142.0,6,NaN,It should be obvious to you what they’re schem...,1.00,6,2023-11-23 22:58:15
kai2hc3,1,If the 🍊💩🤡 gets back in the WH he will immedia...,climate,2023-11-23 23:36:18,18279vq,Big-Summer-,0,1,True,2023-08-21 17:04:57,...,0.0,81.0,8049.0,8140.0,213,NaN,New report says oil and gas sector must slash ...,0.98,213,2023-11-23 18:11:16


In [2]:
df.nunique()

score                           481
self_text                    114731
subreddit                        26
created_time                 115274
post_id                        5507
author_name                   32325
controversiality                  2
ups                             481
user_is_verified                  2
user_account_created_time     31645
user_awardee_karma             3029
user_awarder_karma             2194
user_link_karma                9236
user_comment_karma            21608
user_total_karma              22812
post_score                      708
post_self_text                 1493
post_title                     5104
post_upvote_ratio                95
post_thumbs_ups                 708
post_created_time              5506
dtype: int64

In [3]:
from plotly.express import histogram
histogram(data_frame=df, x='score', log_y=True)

In [4]:
from plotly.express import scatter
scatter(data_frame=df[df['score'] > 40], x='user_link_karma', y='user_comment_karma', color='score', hover_name='author_name',
        log_x=True, log_y=True, trendline='ols')

In [5]:
from plotly.express import bar
TOPN = 50
bar(data_frame=df[df['author_name'] != '[deleted]']['author_name'].value_counts().to_frame().reset_index().head(n=TOPN), x='author_name', y='count',
   title='Top {} authors by comment count'.format(TOPN))

In [6]:
df[df['author_name'] != '[deleted]']['author_name'].value_counts().to_frame().reset_index().head(n=TOPN)['count'].sum() / len(df)

0.11494213281895839

The top 50 commenters out of neary 32000 contribute 15% of the comments.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from plotly.express import treemap

NGRAM_RANGE = (1, 1)
TOPN = 100
all_comment_model = CountVectorizer(encoding='utf-8', stop_words='english', min_df=15, max_df=0.3, lowercase=True, ngram_range=NGRAM_RANGE)
all_comment_result = all_comment_model.fit_transform(raw_documents=df['self_text'])
all_comment_df = pd.DataFrame(data={'word': all_comment_model.get_feature_names_out().tolist(),
                                    'count': all_comment_result.toarray().sum(axis=0).tolist()})
all_comment_df['numeric'] = all_comment_df['word'].str.isnumeric()
print(all_comment_df.shape)
treemap(data_frame=all_comment_df[~all_comment_df['numeric']].sort_values(ascending=False, by='count').head(n=TOPN), path=['word'], names='word', values='count', color='count', height=800,
       color_continuous_scale='bluered', title = '{} {}-grams by count'.format(min(len(all_comment_df), TOPN), NGRAM_RANGE))

(12430, 3)


In [8]:
all_text_model = CountVectorizer(encoding='utf-8', stop_words='english', min_df=4, max_df=0.15, lowercase=True, ngram_range=NGRAM_RANGE)
all_text_result = all_text_model.fit_transform(raw_documents=df.dropna(subset='post_self_text')[['post_self_text']].drop_duplicates(ignore_index=True)['post_self_text'].tolist())
all_text_df = pd.DataFrame(data={'word': all_text_model.get_feature_names_out().tolist(),
                                    'count': all_text_result.toarray().sum(axis=0).tolist()})
all_text_df['numeric'] = all_text_df['word'].str.isnumeric()
print(all_text_df.shape)
treemap(data_frame=all_text_df[~all_text_df['numeric']].sort_values(ascending=False, by='count').head(n=TOPN), path=['word'], names='word', values='count', color='count', height=800,
       color_continuous_scale='bluered', title = '{} {}-grams by count'.format(min(len(all_text_df), TOPN), NGRAM_RANGE))

(4563, 3)


In [9]:
top_comment_tokens = all_comment_df[~all_comment_df['numeric']].sort_values(ascending=False, by='count').head(n=TOPN)['word'].tolist()
top_post_tokens = all_text_df[~all_text_df['numeric']].sort_values(ascending=False, by='count').head(n=TOPN)['word'].tolist()
comment_not_post = [word for word in top_comment_tokens if word not in top_post_tokens]
post_not_comment = [word for word in top_post_tokens if word not in top_comment_tokens]
print(len(post_not_comment), sorted(post_not_comment))

39 ['action', 'articles', 'best', 'crisis', 'current', 'day', 'days', 'different', 'en', 'environment', 'feel', 'gov', 'greenhouse', 'help', 'hope', 'html', 'impact', 'land', 'let', 'looking', 'models', 'natural', 'news', 'noaa', 'ocean', 'possible', 'post', 'question', 'research', 'start', 'temperatures', 'thanks', 'true', 'understand', 'using', 'weather', 'wiki', 'wikipedia', 'x200b']


In [10]:
print(len(comment_not_post), sorted(comment_not_post))

39 ['amp', 'average', 'bad', 'big', 'change', 'climate', 'com', 'did', 'doesn', 'doing', 'don', 'end', 'far', 'fuel', 'fuels', 'global', 'https', 'humans', 'isn', 'just', 'know', 'like', 'maybe', 'mean', 'money', 'oil', 'people', 'real', 'stop', 'sure', 'thing', 'think', 'time', 've', 'won', 'world', 'www', 'years', 'yes']


It is probably not surprising to see a climate discussion feature words like weather, ocean, model, noaa, and fuel and oil; it is probably a little surprising to see that the comments feature fuel/oil prominently and the articles not so much.